# Input

In [133]:
file_path = 'uml.txt'

with open(file_path, 'r') as file:

    uml = file.read()    # Read the file line by line
    # print(uml)


In [134]:
print(uml)

class SHAS
{
	1 <@>- 1..* Room rooms;
	1 <@>- 1 ActivityLog log;
	1 <@>- * AutomationRule allRules;
	// subset of allRules
	1 -> * AutomationRule activeRules;
	// When editing rules, can only retrieve from this set
	// subset of allRules but intersection with activerules = null
	1 -> * AutomationRule inactiveRules; 
}

class ActivityLog
{
	// Maybe composition with SHAS and simple association here?
	1 <@>- * SensorReading sensorReadings;
	1 <@>- * ActuatorCommand commands;
}

// This abstract interface is used for all physical elements of the SHAS
// so that they can be easily referred to by the automation rules as "Elements" of the SHAS.
class Element
{
}

class Room
{
	isA Element;
	1 <@>- * Device devices;
}

class Device
{
	isA Element;
	autounique identifier;
	Boolean isActive;
}

class Sensor
{
	isA Device;
	enum SensorType { Temperature, Movement };
	SensorType type;
	
	1 -- * SensorReading readings;
}

class SensorReading
{
	isA Element;
	Double reading;
	Time timestamp;
	Strin

In [135]:
final_uml = []
for line in uml.splitlines():
    if "//" in line:
        print(line)
        line_no_comment = line.split("//")[0]
        print(f"new line:{line_no_comment}")
        # line_no_comment += "\n"

        # final_uml = final_uml.replace(line, line_no_comment)
        
        final_uml.append(line_no_comment)
    else:
        final_uml.append(line)

uml = "\n".join(final_uml)


	// subset of allRules
new line:	
	// When editing rules, can only retrieve from this set
new line:	
	// subset of allRules but intersection with activerules = null
new line:	
	// Maybe composition with SHAS and simple association here?
new line:	
// This abstract interface is used for all physical elements of the SHAS
new line:
// so that they can be easily referred to by the automation rules as "Elements" of the SHAS.
new line:
// Dealing with the automation rules:
new line:
	// 0..1 in case of NOT, this should be ignored
new line:	
	// Some parser will parse the relations (and atomic terms) and determine the final value of the precondition
new line:	
	// Use a separate instance of a precondition for each automationrule.
new line:	
	// "SHAS records whenever a rule was triggered using a timestamp"
new line:	


In [136]:
print(uml)

class SHAS
{
	1 <@>- 1..* Room rooms;
	1 <@>- 1 ActivityLog log;
	1 <@>- * AutomationRule allRules;
	
	1 -> * AutomationRule activeRules;
	
	
	1 -> * AutomationRule inactiveRules; 
}

class ActivityLog
{
	
	1 <@>- * SensorReading sensorReadings;
	1 <@>- * ActuatorCommand commands;
}



class Element
{
}

class Room
{
	isA Element;
	1 <@>- * Device devices;
}

class Device
{
	isA Element;
	autounique identifier;
	Boolean isActive;
}

class Sensor
{
	isA Device;
	enum SensorType { Temperature, Movement };
	SensorType type;
	
	1 -- * SensorReading readings;
}

class SensorReading
{
	isA Element;
	Double reading;
	Time timestamp;
	String readingType;
}

class Actuator
{
	isA Device;
	enum ActuatorType { Light, Lock };
	ActuatorType type;
	
	1 -- * ActuatorCommand commands;
}

class ActuatorCommand
{
	isA Element;
	enum CommandType { LockDoor, TurnOnHeating, TurnOnLights };
	CommandType type;
	Time timestamp;
	enum CommandStatus { Requested, Completed, Failed };
	CommandStatus status;
}





In [137]:
final_uml = uml
for line in uml.splitlines():
    if "enum" in line:
        print(line.replace("{","(").replace("}", ")").replace("enum","").strip().replace(";", ""))
        final_uml = final_uml.replace(line, "")

uml = final_uml

SensorType ( Temperature, Movement )
ActuatorType ( Light, Lock )
CommandType ( LockDoor, TurnOnHeating, TurnOnLights )
CommandStatus ( Requested, Completed, Failed )
Operator ( AND, OR, NOT)


# Parse

## functions

In [138]:
import re

def extract_content_within_braces(input_string):
    # Define a regular expression pattern to match content within curly braces
    pattern = r'\{([^}]+)\}'

    # Use re.findall to find all matches in the input string
    matches = re.findall(pattern, input_string)

    
    # Return the list of matches
    return matches

# Example usage
input_string = "{bla bla} {example} {another example}"
result = extract_content_within_braces(input_string)

print("Original string:", input_string)
print("Content within braces:", result)

Original string: {bla bla} {example} {another example}
Content within braces: ['bla bla', 'example', 'another example']


In [139]:
def create_nested_list(matchs):
    nested_list = []
    for paragraph in matchs:
        true_line = []
        lines = paragraph.split('\n')
        for i in range(len(lines)):
            lines[i] = lines[i].strip()
            lines[i] = lines[i].split("//")[0].strip()
            if len(lines[i])>0 and not "position" in lines[i]:
                true_line.append(lines[i])

        
        nested_list.append(true_line)

    return nested_list


In [140]:
def find_class(uml):
    ans = []
    lines = uml.splitlines()

    for l in lines:
        if "class" in l:
            cls = l.replace("class","")
            cls = cls.strip()
            if not cls in ans:
                ans.append(cls)
    return ans

In [141]:
def is_rel(line):
    contain = "<@>"
    aso = "--"
    uni_aso_l = "->"
    uni_aso_r = "<-"
    isa = "isA"
    rels = [contain, aso, uni_aso_l, uni_aso_r, isa]

    for rel in rels:
        if rel in line:
            return True
    return False

In [142]:
def process_rel(cls, line):
    line = line.split(";")[0]
    
    contain = "<@>"
    aso = "--"
    uni_aso_l = "->"
    uni_aso_r = "<-"
    isa = "isA"
    if "-<@>" in line:
        elements = line.split("-<@>")
        return elements[1] + " contain " + elements[0] + cls
    if "<@>-" in line:
        elements = line.split("<@>-")
        return elements[0] + cls +  " contain " + elements[1]      

    if "--" in line:
        elements = line.split("--")
        return elements[0] + cls +  " associate " + elements[1]      

    if "->" in line:
        elements = line.split("->")
        return elements[0] + cls +  " associate " + elements[1] 
    
    if "<-" in line:
        elements = line.split("<-")
        return elements[0] + cls +  " associate " + elements[1]      
    
    if isa in line:
        return cls + line.replace("isA", " inherit ")
    
    return "cannot process:" + line

## inference

In [143]:
print(uml)

class SHAS
{
	1 <@>- 1..* Room rooms;
	1 <@>- 1 ActivityLog log;
	1 <@>- * AutomationRule allRules;
	
	1 -> * AutomationRule activeRules;
	
	
	1 -> * AutomationRule inactiveRules; 
}

class ActivityLog
{
	
	1 <@>- * SensorReading sensorReadings;
	1 <@>- * ActuatorCommand commands;
}



class Element
{
}

class Room
{
	isA Element;
	1 <@>- * Device devices;
}

class Device
{
	isA Element;
	autounique identifier;
	Boolean isActive;
}

class Sensor
{
	isA Device;

	SensorType type;
	
	1 -- * SensorReading readings;
}

class SensorReading
{
	isA Element;
	Double reading;
	Time timestamp;
	String readingType;
}

class Actuator
{
	isA Device;

	ActuatorType type;
	
	1 -- * ActuatorCommand commands;
}

class ActuatorCommand
{
	isA Element;

	CommandType type;
	Time timestamp;

	CommandStatus status;
}



class AtomicTerm
{
	String expression;
	1 -- 1..* Element refersTo;
	Boolean atomicValue;
}

class Relation
{

	Operator op;
	1 -> 1 AtomicTerm LHS;
	
	1 -> 0..1 AtomicTerm RHS;
	Boolean valu

In [144]:
classes = find_class(uml)
print(classes)
len(classes)

['SHAS', 'ActivityLog', 'Element', 'Room', 'Device', 'Sensor', 'SensorReading', 'Actuator', 'ActuatorCommand', 'AtomicTerm', 'Relation', 'Precondition', 'AutomationRule']


13

In [145]:
result = extract_content_within_braces(uml)
result = create_nested_list(result)
print(result)
len(result)

[['1 <@>- 1..* Room rooms;', '1 <@>- 1 ActivityLog log;', '1 <@>- * AutomationRule allRules;', '1 -> * AutomationRule activeRules;', '1 -> * AutomationRule inactiveRules;'], ['1 <@>- * SensorReading sensorReadings;', '1 <@>- * ActuatorCommand commands;'], [], ['isA Element;', '1 <@>- * Device devices;'], ['isA Element;', 'autounique identifier;', 'Boolean isActive;'], ['isA Device;', 'SensorType type;', '1 -- * SensorReading readings;'], ['isA Element;', 'Double reading;', 'Time timestamp;', 'String readingType;'], ['isA Device;', 'ActuatorType type;', '1 -- * ActuatorCommand commands;'], ['isA Element;', 'CommandType type;', 'Time timestamp;', 'CommandStatus status;'], ['String expression;', '1 -- 1..* Element refersTo;', 'Boolean atomicValue;'], ['Operator op;', '1 -> 1 AtomicTerm LHS;', '1 -> 0..1 AtomicTerm RHS;', 'Boolean value;'], ['1 -> 1..* Relation relations;', 'Boolean value;'], ['1 -- 1 Precondition precondition;', '1 -- * ActuatorCommand actionSequence;', 'Boolean isActive;

13

## DSL

In [146]:
for cls, atr in zip(classes, result):
    print(cls)
    cls_line = cls + "("
    has_atr = False
    for a in atr:
        print(a)
        
        if not is_rel(a):
            a = a.replace(";", ",")
            cls_line = cls_line + " " + a
            has_atr = True
        else:
            print("-"*10 + " rel below " + "-"*10)
            print(process_rel(cls, a).strip())
            print(" ")

    if has_atr:
        cls_line = cls_line[:-1]
    cls_line += ")"
    print(cls_line.replace("{",""))
    print()
    print( "=" * 40)
    print("\n")

SHAS
1 <@>- 1..* Room rooms;
---------- rel below ----------
1 SHAS contain  1..* Room rooms
 
1 <@>- 1 ActivityLog log;
---------- rel below ----------
1 SHAS contain  1 ActivityLog log
 
1 <@>- * AutomationRule allRules;
---------- rel below ----------
1 SHAS contain  * AutomationRule allRules
 
1 -> * AutomationRule activeRules;
---------- rel below ----------
1 SHAS associate  * AutomationRule activeRules
 
1 -> * AutomationRule inactiveRules;
---------- rel below ----------
1 SHAS associate  * AutomationRule inactiveRules
 
SHAS()



ActivityLog
1 <@>- * SensorReading sensorReadings;
---------- rel below ----------
1 ActivityLog contain  * SensorReading sensorReadings
 
1 <@>- * ActuatorCommand commands;
---------- rel below ----------
1 ActivityLog contain  * ActuatorCommand commands
 
ActivityLog()



Element
Element()



Room
isA Element;
---------- rel below ----------
Room inherit  Element
 
1 <@>- * Device devices;
---------- rel below ----------
1 Room contain  * Device dev